In [10]:
# Import necessary libraries

# Data manipulation
import pandas as pd
import numpy as np

# Statistical functions
from scipy.stats import zscore

# For concurrency (running functions in parallel)
from concurrent.futures import ThreadPoolExecutor

# For caching (to speed up repeated function calls)
from functools import lru_cache

# For progress tracking
from tqdm import tqdm

# Plotting and Visualisation
import matplotlib.pyplot as plt
import seaborn as sns

# Language Detection packages
# `langdetect` for detecting language
from langdetect import detect as langdetect_detect, DetectorFactory
from langdetect.lang_detect_exception import LangDetectException
# `langid` for an alternative language detection method
from langid import classify as langid_classify

# Text Preprocessing and NLP
# Stopwords (common words to ignore) from NLTK
from nltk.corpus import stopwords
# Tokenizing sentences/words
from nltk.tokenize import word_tokenize
# Part-of-speech tagging
from nltk import pos_tag
# Lemmatization (converting words to their base form)
from nltk.stem import WordNetLemmatizer
import nltk
# Regular expressions for text pattern matching
import re

# Word Cloud generation
from wordcloud import WordCloud

In [11]:
import os
import sys
from pathlib import Path

if "workding_dir" not in locals():
    workding_dir = str(Path.cwd().parent)
os.chdir(workding_dir)
sys.path.append(workding_dir)
print("working dir:", workding_dir)

working dir: /home/dariusng2103/projects/dm_project/DM-Fake-News-Detection


In [12]:
from datasets import load_dataset, concatenate_datasets, Dataset

datasets = load_dataset(
    "csv",
    data_files={
        "train": [
            "dataset/train_data_1.csv",
            "dataset/train_data_2.csv",
            "dataset/train_data_3.csv",
            "dataset/train_data_4.csv",
        ],
        "test": "dataset/test_data.csv",
        "rewritten_train": [
            "dataset/rewritten_train_data_1.csv",
            "dataset/rewritten_train_data_2.csv",
            "dataset/rewritten_train_data_3.csv",
            "dataset/rewritten_train_data_4.csv",
        ],
        "rewritten_test": "dataset/rewritten_test_data.csv",
    },
)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
    rewritten_train: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 54441
    })
    rewritten_test: Dataset({
        features: ['label', 'full_content', 'processed_full_content'],
        num_rows: 6050
    })
})

### RNN with Word2Vec (best RNN model)

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from gensim.models import Word2Vec
import tensorflow as tf
import numpy as np
import random

from gensim.models import Word2Vec
import numpy as np

def train_word2vec_embeddings(train_texts, word_index, vocab_size=5000, embedding_dim=128):
    """Tokenizes text, trains Word2Vec, and creates an embedding matrix."""
    sentences = [text.split() for text in train_texts]  # Convert text into word lists

    # Train Word2Vec
    word2vec_model = Word2Vec(sentences=sentences, vector_size=embedding_dim, window=5, min_count=1, workers=4, sg=1)

    # Initialize embedding matrix
    embedding_matrix = np.zeros((vocab_size, embedding_dim))

    for word, i in word_index.items():
        if i < vocab_size and word in word2vec_model.wv:
            embedding_matrix[i] = word2vec_model.wv[word]

    return embedding_matrix


def create_rnn_model(vocab_size, embedding_matrix, input_length, rnn_units=64, dropout_rate=0.5, learning_rate=0.001):
    """Creates an RNN model using Word2Vec embeddings."""
    model = Sequential([
        Embedding(vocab_size, embedding_matrix.shape[1], weights=[embedding_matrix], 
                  input_length=input_length, trainable=True),
        SimpleRNN(rnn_units, activation='tanh'),
        Dropout(dropout_rate),
        Dense(1, activation='sigmoid')
    ])

    model.compile(optimizer=Adam(learning_rate), loss='binary_crossentropy', metrics=['accuracy'])
    return model


In [14]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split

def train_model(train_data, val_data, rnn_units=64, dropout_rate=0.5, epochs=10, batch_size=64, grid_search=False):
    """Trains an RNN model using Word2Vec embeddings."""
    seed = 42
    tf.random.set_seed(seed)
    np.random.seed(seed)

    vocab_size = 5000
    embedding_dim = 128
    max_sequence_length = 300

    print(f"\n🚀 Training RNN with rnn_units={rnn_units}, dropout_rate={dropout_rate}")

    # Extract texts and labels
    train_texts = train_data["processed_full_content"]
    val_texts = val_data["processed_full_content"]
    y_train = train_data["label"]
    y_val = val_data["label"]

    # Tokenize texts
    tokenizer = Tokenizer(num_words=vocab_size)
    tokenizer.fit_on_texts(train_texts)

    # Convert texts to sequences and pad them
    X_train = pad_sequences(tokenizer.texts_to_sequences(train_texts), maxlen=max_sequence_length)
    X_val = pad_sequences(tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length)

    # Train Word2Vec and create embeddings
    embedding_matrix = train_word2vec_embeddings(train_texts, tokenizer.word_index, vocab_size, embedding_dim)

    # Create and compile the RNN model
    model = create_rnn_model(
        vocab_size=vocab_size,
        embedding_matrix=embedding_matrix,
        input_length=max_sequence_length,
        rnn_units=rnn_units,
        dropout_rate=dropout_rate
    )

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    # Train the model
    model.fit(
        X_train, y_train,
        epochs=epochs,
        batch_size=batch_size,
        validation_data=(X_val, y_val),
        callbacks=[early_stopping],
        verbose=1
    )

    # Evaluate model performance on validation data
    y_pred = (model.predict(X_val) > 0.5).astype(int).reshape(-1)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    #  Print evaluation metrics
    result = {"accuracy": accuracy, "precision": precision, "recall": recall, "f1_score": f1}
    print("\n🏆 Training Results:")

    for key, value in result.items():
        print(f"🔹 {key.capitalize()}: {value:.4f}")

    return result if grid_search else model

In [15]:
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
import pandas as pd
from tqdm import tqdm


def evaluate_model(model, train_data, val_data):
    print("Evaluating Model")

    max_words = 10000
    max_sequence_length = 300

    train_texts = train_data["processed_full_content"]
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(train_texts)

    y_val = val_data["label"]
    val_texts = val_data["processed_full_content"]

    X_val = pad_sequences(
        tokenizer.texts_to_sequences(val_texts), maxlen=max_sequence_length
    )
    y_pred = (model.predict(X_val) > 0.5).astype(int)

    accuracy = accuracy_score(y_val, y_pred)
    precision = precision_score(y_val, y_pred)
    recall = recall_score(y_val, y_pred)
    f1 = f1_score(y_val, y_pred)

    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1: {f1:.4f}")

In [16]:
from sklearn.model_selection import StratifiedKFold

def do_grid_search(data):
    """Performs Grid Search for the best RNN hyperparameters."""
    param_grid = {"rnn_units": [32, 64], "dropout_rate": [0.3, 0.5]}
    results = []
    best_score = 0
    best_params = None

    for rnn_units in param_grid["rnn_units"]:
        for dropout_rate in param_grid["dropout_rate"]:
            print(f"\n🔍 Testing rnn_units={rnn_units}, dropout_rate={dropout_rate}")

            kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
            fold_scores = []

            for fold, (train_idx, val_idx) in enumerate(
                kfold.split(data["processed_full_content"], data["label"].to_numpy().ravel()), 1
            ):
                print(f"\n📂 Fold {fold}")

                train_data = data.iloc[train_idx]
                val_data = data.iloc[val_idx]

                result = train_model(train_data, val_data, rnn_units=rnn_units, dropout_rate=dropout_rate, grid_search=True)
                fold_scores.append(result["f1_score"])

            avg_score = np.mean(fold_scores)
            print(f"📊 Average F1-score: {avg_score:.4f}")

            if avg_score > best_score:
                best_score = avg_score
                best_params = {"rnn_units": rnn_units, "dropout_rate": dropout_rate}

    return best_params


In [17]:
train_data = datasets["train"].to_pandas()
val_data = datasets["test"].to_pandas()
data = pd.concat([train_data, val_data], ignore_index=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60491 entries, 0 to 60490
Data columns (total 3 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   label                   60491 non-null  int64 
 1   full_content            60491 non-null  object
 2   processed_full_content  60491 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.4+ MB


In [18]:
%%time
best_params = do_grid_search(data)



🔍 Testing rnn_units=32, dropout_rate=0.3

📂 Fold 1

🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.8396 - loss: 0.3553 - val_accuracy: 0.9229 - val_loss: 0.2220
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.9396 - loss: 0.1843 - val_accuracy: 0.9499 - val_loss: 0.1529
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9413 - loss: 0.1727 - val_accuracy: 0.9451 - val_loss: 0.1600
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.9497 - loss: 0.1465 - val_accuracy: 0.9453 - val_loss: 0.1693
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.9633 - loss: 0.1176 - val_accuracy: 0.9512 - val_loss: 0.1615
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step

🏆 Training Results:
🔹 Accuracy: 0.9499
🔹 Precision: 0.9354
🔹 Recall: 0.9513
🔹 F1_score: 0.9432

📂 Fold 2

🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 16s 20ms/step - accuracy: 0.8382 - loss: 0.3672 - val_accuracy: 0.9066 - val_loss: 0.2520
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9376 - loss: 0.1896 - val_accuracy: 0.9435 - val_loss: 0.1553
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.9481 - loss: 0.1521 - val_accuracy: 0.9327 - val_loss: 0.1893
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 13s 17ms/step - accuracy: 0.9473 - loss: 0.1543 - val_accuracy: 0.9318 - val_loss: 0.2300
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.9524 - loss: 0.1471 - val_accuracy: 0.9410 - val_loss: 0.1886
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step

🏆 Training Results:
🔹 Accuracy: 0.9435
🔹 Precision: 0.9395
🔹 Recall: 0.9307
🔹 F1_score: 0.9351

📂 Fold 3

🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


...

757/757 ━━━━━━━━━━━━━━━━━━━━ 19s 23ms/step - accuracy: 0.8624 - loss: 0.3213 - val_accuracy: 0.9386 - val_loss: 0.1797
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.9476 - loss: 0.1599 - val_accuracy: 0.8946 - val_loss: 0.2562
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.9528 - loss: 0.1373 - val_accuracy: 0.9388 - val_loss: 0.1661
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.9632 - loss: 0.1106 - val_accuracy: 0.9372 - val_loss: 0.2083
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.8629 - loss: 0.3141 - val_accuracy: 0.9435 - val_loss: 0.1873
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.9541 - loss: 0.1407 - val_accuracy: 0.9364 - val_loss: 0.1883
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step

🏆 Training Results:
🔹 Accuracy: 0.9388
🔹 Precision: 0.9191
🔹 Recall: 0.9429
🔹 F1_score: 0.9309

📂 Fold 4

🚀 Training RNN with rnn_units=64, dropout_rate=0.5
Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 19s 23ms/step - accuracy: 0.8694 - loss: 0.2995 - val_accuracy: 0.9428 - val_loss: 0.1651
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.9506 - loss: 0.1495 - val_accuracy: 0.9214 - val_loss: 0.2170
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.9515 - loss: 0.1454 - val_accuracy: 0.9466 - val_loss: 0.1653
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 15s 19ms/step - accuracy: 0.9185 - loss: 0.2177 - val_accuracy: 0.9302 - val_loss: 0.1986
379/379 ━━━━━━━━━━━━━━━━━━━━ 4s 12ms/step

🏆 Training Results:
🔹 Accuracy: 0.9428
🔹 Precision: 0.9302
🔹 Recall: 0.9397
🔹 F1_score: 0.9350

📂 Fold 5

🚀 Training RNN with rnn_units=64, dropout_rate=0.5
Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


757/757 ━━━━━━━━━━━━━━━━━━━━ 17s 21ms/step - accuracy: 0.8551 - loss: 0.3252 - val_accuracy: 0.9249 - val_loss: 0.2158
Epoch 2/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.9441 - loss: 0.1605 - val_accuracy: 0.9348 - val_loss: 0.1817
Epoch 3/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 14s 19ms/step - accuracy: 0.9408 - loss: 0.1633 - val_accuracy: 0.9316 - val_loss: 0.1799
Epoch 4/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 16s 21ms/step - accuracy: 0.9487 - loss: 0.1505 - val_accuracy: 0.9328 - val_loss: 0.1981
Epoch 5/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.9654 - loss: 0.1045 - val_accuracy: 0.9421 - val_loss: 0.1745
Epoch 6/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.9701 - loss: 0.0908 - val_accuracy: 0.9398 - val_loss: 0.1865
Epoch 7/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 15s 20ms/step - accuracy: 0.9665 - loss: 0.0990 - val_accuracy: 0.9415 - val_loss: 0.2006
Epoch 8/10
757/757 ━━━━━━━━━━━━━━━━━━━━ 16s 22ms/step - accuracy: 0.9772 - loss: 0.0688 - val_accurac

In [20]:
%%time
model = train_model(train_data, val_data, rnn_units=best_params["rnn_units"], dropout_rate=best_params["dropout_rate"])



🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


851/851 ━━━━━━━━━━━━━━━━━━━━ 17s 19ms/step - accuracy: 0.8364 - loss: 0.3494 - val_accuracy: 0.9450 - val_loss: 0.1645
Epoch 2/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - accuracy: 0.9502 - loss: 0.1520 - val_accuracy: 0.9496 - val_loss: 0.1555
Epoch 3/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 15s 17ms/step - accuracy: 0.9559 - loss: 0.1355 - val_accuracy: 0.9428 - val_loss: 0.1708
Epoch 4/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 16s 19ms/step - accuracy: 0.9548 - loss: 0.1392 - val_accuracy: 0.9431 - val_loss: 0.1897
Epoch 5/10
851/851 ━━━━━━━━━━━━━━━━━━━━ 14s 17ms/step - accuracy: 0.9661 - loss: 0.1035 - val_accuracy: 0.9428 - val_loss: 0.1725
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step

🏆 Training Results:
🔹 Accuracy: 0.9496
🔹 Precision: 0.9308
🔹 Recall: 0.9558
🔹 F1_score: 0.9432
CPU times: user 7min 9s, sys: 14.9 s, total: 7min 24s
Wall time: 3min 14s


In [21]:
model.save("results/RNN_model_original.keras")

In [22]:
# load model
from tensorflow.keras.models import load_model

model2 = load_model("results/RNN_model_original.keras")
model2.summary()

Model: "sequential_22"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_22 (Embedding)        │ (None, 300, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_22 (SimpleRNN)       │ (None, 32)             │         5,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_22 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,935,557 (7.38 MB)

 Trainable params: 645,185 (2.46 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,290,372 (4.92 MB)

In [23]:
%%time
evaluate_model(model, train_data, val_data)


Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step
Accuracy: 0.9479
Precision: 0.9262
Recall: 0.9573
F1: 0.9415
CPU times: user 9.9 s, sys: 443 ms, total: 10.3 s
Wall time: 9.31 s


In [24]:
%%time
evaluate_model(model2, train_data, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
Accuracy: 0.9479
Precision: 0.9262
Recall: 0.9573
F1: 0.9415
CPU times: user 10.2 s, sys: 354 ms, total: 10.5 s
Wall time: 11.1 s


In [25]:
val_data_rewritten = datasets["rewritten_test"].to_pandas()
train_data_rewritten = datasets["rewritten_train"].to_pandas()
data_rewritten = pd.concat([train_data, train_data_rewritten, val_data, val_data_rewritten], ignore_index=True)
data_rewritten.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120982 entries, 0 to 120981
Data columns (total 3 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   label                   120982 non-null  int64 
 1   full_content            120982 non-null  object
 2   processed_full_content  120982 non-null  object
dtypes: int64(1), object(2)
memory usage: 2.8+ MB


In [26]:
%%time

evaluate_model(model, train_data, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Accuracy: 0.7621
Precision: 0.7606
Recall: 0.6660
F1: 0.7102
CPU times: user 9.52 s, sys: 367 ms, total: 9.88 s
Wall time: 10.3 s


In [27]:
%%time

evaluate_model(model2, train_data, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step
Accuracy: 0.7621
Precision: 0.7606
Recall: 0.6660
F1: 0.7102
CPU times: user 10.5 s, sys: 338 ms, total: 10.8 s
Wall time: 9.78 s


In [28]:
%%time

best_params_rewritten = do_grid_search(data_rewritten)
best_params_rewritten


🔍 Testing rnn_units=32, dropout_rate=0.3

📂 Fold 1

🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - accuracy: 0.7992 - loss: 0.4446 - val_accuracy: 0.7539 - val_loss: 0.4865
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 29s 19ms/step - accuracy: 0.7295 - loss: 0.5118 - val_accuracy: 0.8742 - val_loss: 0.3264
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8226 - loss: 0.4076 - val_accuracy: 0.8683 - val_loss: 0.3389
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8631 - loss: 0.3490 - val_accuracy: 0.8597 - val_loss: 0.3481
Epoch 5/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8587 - loss: 0.3371 - val_accuracy: 0.8750 - val_loss: 0.3273
757/757 ━━━━━━━━━━━━━━━━━━━━ 7s 10ms/step

🏆 Training Results:
🔹 Accuracy: 0.8742
🔹 Precision: 0.8468
🔹 Recall: 0.8699
🔹 F1_score: 0.8582

📂 Fold 2

🚀 Training RNN with rnn_units=32, dropout_rate=0.3
Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 33s 21ms/step - accuracy: 0.8047 - loss: 0.4220 - val_accuracy: 0.7996 - val_loss: 0.4390
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 31s 21ms/step - accuracy: 0.8619 - loss: 0.3447 - val_accuracy: 0.8686 - val_loss: 0.3225
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8667 - loss: 0.3234 - val_accuracy: 0.8735 - val_loss: 0.3382
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8517 - loss: 0.3620 - val_accuracy: 0.8902 - val_loss: 0.2858
Epoch 5/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8543 - loss: 0.3688 - val_accuracy: 0.8678 - val_loss: 0.3369
Epoch 6/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8710 - loss: 0.3339 - val_accuracy: 0.8799 - val_loss: 0.2895
Epoch 7/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 30s 20ms/step - accuracy: 0.8601 - loss: 0.3415 - val_accuracy: 0.8852 - val_loss: 0.3202
757/757 ━━━━━━━━━━━━━━━━━━━━ 6s 8ms/step

🏆 Training Results:
🔹 Accuracy: 0.8902
🔹 Prec

...

/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 34s 22ms/step - accuracy: 0.7778 - loss: 0.4781 - val_accuracy: 0.8845 - val_loss: 0.3189
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.8142 - loss: 0.4198 - val_accuracy: 0.8789 - val_loss: 0.3412
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 33s 22ms/step - accuracy: 0.8084 - loss: 0.4294 - val_accuracy: 0.8474 - val_loss: 0.3730
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.8660 - loss: 0.3483 - val_accuracy: 0.8866 - val_loss: 0.3114
Epoch 5/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.8241 - loss: 0.3998 - val_accuracy: 0.8571 - val_loss: 0.3518
Epoch 6/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.8817 - loss: 0.3190 - val_accuracy: 0.8905 - val_loss: 0.2868
Epoch 7/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.8519 - loss: 0.3492 - val_accuracy: 0.8133 - val_loss: 0.4334
Epoch 8/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.8569 - loss: 0.35

/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1513/1513 ━━━━━━━━━━━━━━━━━━━━ 34s 22ms/step - accuracy: 0.7962 - loss: 0.4486 - val_accuracy: 0.8375 - val_loss: 0.3996
Epoch 2/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.8382 - loss: 0.3921 - val_accuracy: 0.8776 - val_loss: 0.3048
Epoch 3/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.7563 - loss: 0.4870 - val_accuracy: 0.8673 - val_loss: 0.3485
Epoch 4/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.8329 - loss: 0.3971 - val_accuracy: 0.8236 - val_loss: 0.4254
Epoch 5/10
1513/1513 ━━━━━━━━━━━━━━━━━━━━ 32s 21ms/step - accuracy: 0.8326 - loss: 0.4031 - val_accuracy: 0.8539 - val_loss: 0.3480
757/757 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step

🏆 Training Results:
🔹 Accuracy: 0.8776
🔹 Precision: 0.8664
🔹 Recall: 0.8516
🔹 F1_score: 0.8589
📊 Average F1-score: 0.8640
CPU times: user 3h 44min 54s, sys: 12min 39s, total: 3h 57min 33s
Wall time: 2h 4min 36s


{'rnn_units': 32, 'dropout_rate': 0.5}

In [29]:
%%time

train_data_combined = pd.concat([train_data, train_data_rewritten], ignore_index=True)
val_data_combined = pd.concat([val_data, val_data_rewritten], ignore_index=True)
model_combined = train_model(train_data_combined, val_data_combined, 
                             rnn_units=best_params_rewritten["rnn_units"], 
                             dropout_rate=best_params_rewritten["dropout_rate"])
model_combined.save("results/RNN_model_combined.keras")


🚀 Training RNN with rnn_units=32, dropout_rate=0.5
Epoch 1/10


/home/dariusng2103/projects/dm_project/fake-news/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1702/1702 ━━━━━━━━━━━━━━━━━━━━ 32s 18ms/step - accuracy: 0.8109 - loss: 0.4281 - val_accuracy: 0.8810 - val_loss: 0.3153
Epoch 2/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 30s 17ms/step - accuracy: 0.8814 - loss: 0.3246 - val_accuracy: 0.8860 - val_loss: 0.2973
Epoch 3/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 29s 17ms/step - accuracy: 0.8854 - loss: 0.3054 - val_accuracy: 0.8888 - val_loss: 0.3114
Epoch 4/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 30s 18ms/step - accuracy: 0.8301 - loss: 0.3875 - val_accuracy: 0.6912 - val_loss: 0.5769
Epoch 5/10
1702/1702 ━━━━━━━━━━━━━━━━━━━━ 29s 17ms/step - accuracy: 0.7195 - loss: 0.5247 - val_accuracy: 0.8707 - val_loss: 0.3252
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step

🏆 Training Results:
🔹 Accuracy: 0.8860
🔹 Precision: 0.9025
🔹 Recall: 0.8291
🔹 F1_score: 0.8642
CPU times: user 11min 50s, sys: 28.3 s, total: 12min 18s
Wall time: 5min 44s


In [30]:
evaluate_model(model_combined, train_data_combined, val_data_combined)

Evaluating Model
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 8ms/step
Accuracy: 0.8844
Precision: 0.8998
Recall: 0.8279
F1: 0.8624


In [31]:
evaluate_model(model_combined, train_data_combined, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Accuracy: 0.9319
Precision: 0.9388
Recall: 0.9033
F1: 0.9207


In [32]:
evaluate_model(model_combined, train_data_combined, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Accuracy: 0.8369
Precision: 0.8571
Recall: 0.7526
F1: 0.8014


In [33]:
model_combined2 = load_model("results/RNN_model_combined.keras")
model_combined2.summary()

Model: "sequential_43"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_43 (Embedding)        │ (None, 300, 128)       │       640,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn_43 (SimpleRNN)       │ (None, 32)             │         5,152 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_43 (Dropout)            │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_43 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,935,557 (7.38 MB)

 Trainable params: 645,185 (2.46 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,290,372 (4.92 MB)

In [34]:
evaluate_model(model_combined2, train_data_combined, val_data_combined)

Evaluating Model
379/379 ━━━━━━━━━━━━━━━━━━━━ 3s 9ms/step
Accuracy: 0.8844
Precision: 0.8998
Recall: 0.8279
F1: 0.8624


In [35]:
evaluate_model(model_combined2, train_data_combined, val_data)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Accuracy: 0.9319
Precision: 0.9388
Recall: 0.9033
F1: 0.9207


In [36]:
evaluate_model(model_combined2, train_data_combined, val_data_rewritten)

Evaluating Model
190/190 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step
Accuracy: 0.8369
Precision: 0.8571
Recall: 0.7526
F1: 0.8014
